In [2]:
# California Housing Advanced ML Pipeline con Stacking e Optuna (CPU-based)
# Obiettivo: Ridurre l'RMSE a <= 0.32 sfruttando l'Ensemble Stacking.

## 0. Installazione Librerie Aggiuntive (Necessaria per LightGBM e CatBoost)
import sys
import subprocess
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "lightgbm", "catboost", "shap", "--quiet"])
except Exception as e:
    print(f"Attenzione: Impossibile installare LightGBM/CatBoost/SHAP: {e}")

## import delle librerie
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost.callback import EarlyStopping
import shap
import warnings
# Configurazione
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## 1. Caricamento e Preparazione del Dataset
print("--- 1. Caricamento e Pulizia del Dataset ---")
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns = housing.feature_names)
df['med_house_val'] = housing.target

# Pulizia e Rinominazione delle colonne
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df = df.rename(columns={
    "medinc": "med_inc",
    "houseage": "house_age",
    "averooms": "ave_rooms",
    "avebedrms": "ave_bedrms",
    "aveoccup": "ave_occup",
})

# Outlier Detection (Metodo IQR, k=1.5)
num_features = ['med_inc', 'house_age', 'ave_rooms', 'ave_bedrms', 'population', 'ave_occup', 'med_house_val']

def get_outlier_mask(dataframe, cols=num_features, k=1.5):
    mask = pd.DataFrame(False, index=dataframe.index, columns=cols)
    for col in cols:
        q1 = dataframe[col].quantile(0.25)
        q3 = dataframe[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - k*iqr
        upper = q3 + k*iqr
        mask[col] = (dataframe[col] < lower) | (dataframe[col] > upper)
    return mask.any(axis=1)

before_count = df.shape[0]
mask = get_outlier_mask(df)
df = df[~mask].dropna()
after_count = df.shape[0]

print(f"Righe dopo la rimozione degli outlier: {after_count} (rimossi {before_count - after_count})")


## 2. Feature Engineering
print("\n--- 2. Feature Engineering Avanzato ---")
# Punti di ancoraggio per il fit quadratico (Curva della Costa)
_LAT = np.array([41.8, 39.0, 37.7, 36.6, 34.4, 34.0, 32.7])
_LON = np.array([-124.2, -123.7, -122.5, -121.9, -120.3, -118.5, -117.2])
_COAST_COEFFS = np.polyfit(_LAT, _LON, deg=2)

def _coast_lon(lat):
    a, b, c = _COAST_COEFFS
    return a*lat*lat + b*lat + c

def classify_location(lat, lon):
    coast_at_lat = _coast_lon(lat)
    coast_buffer = 1
    inland_shift = 2.3
    inland_at_lat = coast_at_lat + inland_shift

    if lon <= coast_at_lat + coast_buffer:
        return "coast"
    elif lon >= inland_at_lat:
        return "inland"
    else:
        return "middle"

df["position"] = df.apply(lambda row: classify_location(row['latitude'], row['longitude']), axis=1)

# Distanza Longitudinale dalla Costa
df["lon_from_coast"] = df["longitude"] - df.apply(lambda row: _coast_lon(row['latitude']), axis=1)

# Rapporti e Densità
df["bedrooms_per_room"] = df["ave_bedrms"] / df["ave_rooms"]
df["rooms_per_house"] = df["ave_rooms"]
df["pop_per_house"] = df["ave_occup"]

# Popolazione per Cella (Densità Locale)
def add_population_per_cell(df, lat_col="latitude", lon_col="longitude", pop_col="population",
                            lat_bin=0.1, lon_bin=0.1):
    df = df.copy()
    df["_lat_bin"] = (df[lat_col] // lat_bin) * lat_bin
    df["_lon_bin"] = (df[lon_col] // lon_bin) * lon_bin
    
    pop_map = (
        df.groupby(["_lat_bin", "_lon_bin"])[pop_col]
        .sum()
        .reset_index()
        .rename(columns={pop_col: "pop_per_cell"})
    )
    
    df = df.merge(pop_map, on=["_lat_bin", "_lon_bin"], how="left")
    return df.drop(columns=["_lat_bin", "_lon_bin"])

df = add_population_per_cell(df)
print(f"Nuove Feature Aggiunte: lon_from_coast, bedrooms_per_room, pop_per_house, pop_per_cell, position")


## 3. Preprocessing: Standardizzazione e Split
print("\n--- 3. Preprocessing: Standardizzazione e Split ---")
X = df.drop(columns = ['med_house_val'])
y = df["med_house_val"]

# Prepariamo i dati per lo Stacking (non è richiesta la standardizzazione per i Tree Models,
# ma è meglio farla per il Meta-Learner se fosse una Regressione Lineare e per coerenza con il tuo codice precedente)
X_numeric = X.drop(columns=['position'], errors='ignore')

# Standardizzazione
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)
X_scaled_df = pd.DataFrame(X_scaled, columns=X_numeric.columns)
y = df["med_house_val"].reset_index(drop=True) 

# Divisione train/test (80-20)
# Usiamo i dati non standardizzati per lo Stacking (i Tree Models funzionano meglio sulle feature native)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_numeric, y, test_size=0.2, random_state=42
)
print(f"Train set (Full): {len(X_train_full)} campioni | Test set: {len(X_test)} campioni")

# Split Secondario per Optuna
X_opt_train, X_opt_val, y_opt_train, y_opt_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=42 # 25% del 80% = 20% totale
)


## 4. Ottimizzazione Iperparametri con Optuna (Light - 20 prove per modello)
print("\n--- 4. Ottimizzazione Iperparametri (20 trial per modello) ---")

# --- XGBoost Objective ---
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 600, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'n_jobs': -1,
        'random_state': 42,
        'objective': 'reg:squarederror'
    }
    model = XGBRegressor(**params)
    model.fit(X_opt_train, y_opt_train, verbose=False)
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

# --- CatBoost Objective ---
def objective_cat(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 600, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'verbose': 0,
        'allow_writing_files': False,
        'random_state': 42
    }
    model = CatBoostRegressor(**params)
    model.fit(X_opt_train, y_opt_train)
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

# --- LightGBM Objective ---
def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 600, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'n_jobs': -1,
        'verbose': -1,
        'random_state': 42
    }
    model = LGBMRegressor(**params)
    model.fit(X_opt_train, y_opt_train)
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

# Esecuzione Ottimizzazione
study_xgb = optuna.create_study(direction='minimize').optimize(objective_xgb, n_trials=20)
study_cat = optuna.create_study(direction='minimize').optimize(objective_cat, n_trials=20)
study_lgbm = optuna.create_study(direction='minimize').optimize(objective_lgbm, n_trials=20)

# Raccolta dei migliori parametri
best_xgb_params = study_xgb.best_params.copy()
best_xgb_params.update({'n_jobs': -1, 'random_state': 42, 'objective': 'reg:squarederror'})

best_cat_params = study_cat.best_params.copy()
best_cat_params.update({'verbose': 0, 'allow_writing_files': False, 'random_state': 42})

best_lgbm_params = study_lgbm.best_params.copy()
best_lgbm_params.update({'n_jobs': -1, 'verbose': -1, 'random_state': 42})

print(f"\nBest RMSE Val -> XGB: {study_xgb.best_value:.4f} | CAT: {study_cat.best_value:.4f} | LGBM: {study_lgbm.best_value:.4f}")

## 5. Stacking Regressor: Level 0 e Level 1
print("\n--- 5. Costruzione Stacking Regressor ---")

# Definizione Level 0 (Base Models)
estimators = [
    ('xgb', XGBRegressor(**best_xgb_params)),
    ('cat', CatBoostRegressor(**best_cat_params)),
    ('lgbm', LGBMRegressor(**best_lgbm_params))]

# Definizione Level 1 (Meta Model)
meta_learner = RidgeCV(alphas=[0.1, 1.0, 10.0]) # Ridge con Cross-Validation per trovare il miglior alpha

stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_learner,
    cv=5,            # 5-Fold per generare previsioni out-of-fold robuste
    n_jobs=-1,
    passthrough=False # Il Meta-Learner prende in input SOLO le predizioni Level 0
)

print("Addestramento Stacking in corso...")
stacking_regressor.fit(X_train_full, y_train_full) # Fit sul training set completo


## 6. Valutazione Finale Stacking Ensemble
print("\n--- 6. Valutazione Finale Ensemble ---")

preds_stacking = stacking_regressor.predict(X_test)
rmse_stacking = np.sqrt(mean_squared_error(y_test, preds_stacking))
r2_stacking = r2_score(y_test, preds_stacking)

# Aggiungi a results (per coerenza con il codice precedente)
results.append({'Modello': 'Ensemble (Stacking)', 'R²': r2_stacking, 'RMSE': rmse_stacking, 'MAE': mean_absolute_error(y_test, preds_stacking), 'MSE': mean_squared_error(y_test, preds_stacking)})


print("-" * 40)
print(f"STACKING FINAL RMSE: {rmse_stacking:.4f}")
print(f"STACKING FINAL R2:   {r2_stacking:.4f}")
print("-" * 40)
print(f"Obiettivo RMSE <= 0.32: {'Sì' if rmse_stacking <= 0.32 else 'No'}")


# Ispezione Pesi Meta-Learner (Importanza)
print("\nPesi Assegnati dal Meta-Modello (Level 1):")
model_names = [name.upper() for name, _ in estimators]
coeffs = stacking_regressor.final_estimator_.coef_
for name, coef in zip(model_names, coeffs):
    print(f"  {name:<10}: {coef:.4f}")


## 7. Visualizzazione Finale e Confronto (Aggiornato)
print('\n--- 7. CONFRONTO FINALE DEI MODELLI ---')
results_df = pd.DataFrame(results).drop_duplicates(subset=['Modello'], keep='last').sort_values(by='R²', ascending=False)
print(results_df)

# Hexbin visualization of STACKING predictions
fig, ax = plt.subplots(figsize=(10,8))
hb = ax.hexbin(y_test, preds_stacking, gridsize=30, cmap='viridis', mincnt=1)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
cb = plt.colorbar(hb, ax=ax)
cb.set_label('Densità Campioni')
ax.set_xlabel('Prezzo Reale')
ax.set_ylabel('Prezzo Predetto')
ax.set_title(f'Predizioni Stacking Ensemble\nR²={r2_stacking:.4f}, RMSE={rmse_stacking:.4f}')
plt.show()

print('\nPipeline completata con Feature Engineering avanzata e Stacking Ensemble ottimizzato.')

--- 1. Caricamento e Pulizia del Dataset ---
Righe dopo la rimozione degli outlier: 16312 (rimossi 4328)

--- 2. Feature Engineering Avanzato ---
Nuove Feature Aggiunte: lon_from_coast, bedrooms_per_room, pop_per_house, pop_per_cell, position

--- 3. Preprocessing: Standardizzazione e Split ---
Train set (Full): 13049 campioni | Test set: 3263 campioni

--- 4. Ottimizzazione Iperparametri (20 trial per modello) ---


[W 2025-12-09 16:54:04,992] Trial 19 failed with parameters: {'n_estimators': 1040, 'learning_rate': 0.028759961176147535, 'max_depth': 9, 'subsample': 0.7695337626923384, 'colsample_bytree': 0.7263155741070259} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Rosy\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Rosy\AppData\Local\Temp\ipykernel_10348\1307820652.py", line 174, in objective_xgb
    model.fit(X_opt_train, y_opt_train, verbose=False)
  File "c:\Users\Rosy\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py", line 774, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\Rosy\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\sklearn.py", line 1368, in fit
    self._Booster = train(
                    ^^^^^^
 

KeyboardInterrupt: 